In [35]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import polars as pl
import pandas as pd
pd.set_option('display.float_format', lambda x: f'{x :.1f}')
from cleaner import DataCleaner
import numpy as np
from datetime import datetime
from tools import (
    order_and_rename,
    import_datasets
)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
movies = pl.read_csv('clean_datasets/movies.csv')
ratings = pl.read_csv('movies_datasets/title_ratings.tsv', separator = "\t", ignore_errors = True)

joined = movies.join(
        ratings,
        left_on = "titre_id",
        right_on = "tconst"
    )

In [11]:
col_to_keep = [
    "titre_id",
    "titre_str",
    "titre_date_sortie",
    "titre_duree",
    "titre_genres",
    "person_id",
    "person_name",
    "person_birthdate",
    "person_job",
    "averageRating",
    "numVotes"
]

col_to_change = [
    "titre_id",
    "titre_str",
    "titre_date_sortie",
    "titre_duree",
    "titre_genres",
    "person_id",
    "person_name",
    "person_birthdate",
    "person_job",
    "rating_avg",
    "rating_votes"
]

df = order_and_rename(joined, col_to_keep, col_to_change)

In [21]:
path = "./clean_datasets/movies_ratings.csv"

df.write_csv("./clean_datasets/movies_ratings.csv")

In [25]:
rating = import_datasets(path, 'pandas', sep = ',')

2023-10-21 15:32:09 INFO     Pandas loaded ! Importing movies_ratings.csv...


In [28]:
cleaning = DataCleaner()
# d'abord je fix l'encodage et apres je clean les \N

clean_encode = cleaning.fix_values(rating, "fix_encode")
clean_n = cleaning.fix_values(clean_encode, "fix_n")
# maintenant je clean les porns ! et supprime les ligne ou nous n'avons pas de type de genre.
clean_porn = cleaning.clean_porn(clean_n, )

print(f"Cleaned {len(rating) - len(clean_porn)} rows")

rating = clean_porn

2023-10-21 15:33:06 INFO     Fixing encoding values...
2023-10-21 15:33:10 INFO     Fixing fucked up values...
2023-10-21 15:33:12 INFO     Cleaning porn movies...


Diif 116227


In [30]:
dates = ["<1900", ">1900", ">1930", ">1960", ">1990", ">2010"]

rating["cuts"] = pd.cut(
    rating["titre_date_sortie"],
    bins=[0, 1900, 1931, 1961, 1991, 2011, 2030],
    labels=dates
)

In [37]:
rating["rating_avg"].median()

6.2

In [36]:
rating["rating_avg"].describe()

count   2527386.0
mean          6.1
std           1.3
min           1.0
25%           5.3
50%           6.2
75%           7.0
max          10.0
Name: rating_avg, dtype: float64

In [48]:
condi = (
    rating["rating_avg"] == 10
)

best_movies = rating[condi]

best_movies

titre_id                 titre_str  titre_date_sortie titre_duree  \
1402872  tt10449358                   Kaputol             2019.0         120   
1402873  tt10449358                   Kaputol             2019.0         120   
1402874  tt10449358                   Kaputol             2019.0         120   
1402875  tt10449358                   Kaputol             2019.0         120   
1402876  tt10449358                   Kaputol             2019.0         120   
...             ...                       ...                ...         ...   
2606129   tt9080346  Poets Are the Destroyers             2021.0           0   
2606130   tt9080346  Poets Are the Destroyers             2021.0           0   
2606131   tt9080346  Poets Are the Destroyers             2021.0           0   
2606132   tt9080346  Poets Are the Destroyers             2021.0           0   
2606133   tt9080346  Poets Are the Destroyers             2021.0           0   

        titre_genres  person_id               person_name person_birthdate  \
1402872        Drama  nm2921912                 Popo Diaz                0   
1402873        Drama  nm0317737                Cherie Gil             1963   
1402874        Drama  nm1368756             Alfred Vargas             1981   
1402875        Drama  nm6651713           Ronwaldo Martin                0   
1402876        Drama  nm0032629              Angel Aquino             1973   
...              ...        ...                       ...              ...   
2606129        Drama  nm5138377                 Nancy Pop                0   
2606130        Drama  nm3530768              J.C. Hopkins                0   
2606131        Drama  nm9126120  Christopher Reza Tabassi                0   
2606132        Drama  nm8279090             Sam T. Wilson                0   
2606133        Drama  nm9877320               Daniel Card                0   

                  person_job  rating_avg  rating_votes   cuts  
1402872  production_designer        10.0             7  >2010  
1402873              actress        10.0             7  >2010  
1402874                actor        10.0             7  >2010  
1402875                actor        10.0             7  >2010  
1402876              actress        10.0             7  >2010  
...                      ...         ...           ...    ...  
2606129             director        10.0             6  >2010  
2606130               writer        10.0             6  >2010  
2606131             producer        10.0             6  >2010  
2606132      cinematographer        10.0             6  >2010  
2606133               editor        10.0             6  >2010  

[632 rows x 12 columns]

In [54]:
for date in dates[1:]:
    condi = (
        (rating["cuts"] == date)
    )


    print(f"Average note for {date}, {rating['rating_avg'][condi].median()}")

Average note for >1900, 6.1
Average note for >1930, 6.2
Average note for >1960, 6.1
Average note for >1990, 6.2
Average note for >2010, 6.3


In [57]:
condi = rating["rating_avg"] > 9

rating["titre_genres"][condi].mode()

0    Documentary
Name: titre_genres, dtype: object